In [1]:
import lerobot
from lerobot.datasets.lerobot_dataset import LeRobotDataset, LeRobotDatasetMetadata
import torch
import numpy as np
from huggingface_hub import HfApi
from IPython.display import display, Video, Image

/workspace/gr00t/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# !python -m lerobot.datasets.v30.convert_dataset_v21_to_v30 --repo-id=LSY-lab/stack_without_ft_tact_v4 --root=/root/.cache/huggingface/lerobot --push-to-hub=false

In [4]:
repo_id = "LSY-lab/stack_without_ft_tact_v4"
ds_metadata = LeRobotDatasetMetadata(repo_id)
ds_metadata

LeRobotDatasetMetadata({
    Repository ID: 'LSY-lab/stack_without_ft_tact_v4',
    Total episodes: '51',
    Total frames: '12244',
    Features: '['observation.images.primary', 'observation.images.wrist', 'observation.state.cartesian', 'observation.state.gripper', 'observation.state.joints', 'observation.state.target', 'observation.state', 'action', 'timestamp', 'frame_index', 'episode_index', 'index', 'task_index']',
})',

In [5]:
dataset = LeRobotDataset(repo_id, episodes=[0, 10, 11, 23])

In [6]:
dataloader = torch.utils.data.DataLoader(
        dataset,
        num_workers=4,
        batch_size=1,
        shuffle=True,
    )

In [11]:
batch = next(iter(dataloader))  # (batch, data_dim)

In [12]:
for key in batch.keys():
    try:
        print(f"{key}: {batch[key].shape}")
    except:
        print(f"Key: {key} has no shape")
        print(batch[key])

observation.images.primary: torch.Size([1, 3, 256, 256])
observation.images.wrist: torch.Size([1, 3, 256, 256])
observation.state.cartesian: torch.Size([1, 6])
observation.state.gripper: torch.Size([1])
observation.state.joints: torch.Size([1, 7])
observation.state.target: torch.Size([1, 6])
observation.state: torch.Size([1, 20])
action: torch.Size([1, 7])
timestamp: torch.Size([1])
frame_index: torch.Size([1])
episode_index: torch.Size([1])
index: torch.Size([1])
task_index: torch.Size([1])
Key: task has no shape
['pick the lego block.']


In [ ]:
batch[""]

tensor([2563])

In [28]:
episode_index = 3
from_idx = dataset.meta.episodes["dataset_from_index"][episode_index]
to_idx = dataset.meta.episodes["dataset_to_index"][episode_index]

In [29]:
from_idx, to_idx

(725, 985)

In [ ]:
camera_key = dataset.meta.camera_keys[0]
frames = [dataset[idx][camera_key] for idx in range(from_idx, to_idx)]

In [ ]:
#  CUDA_VISIBLE_DEVICES=0 python     gr00t/experiment/launch_finetune.py     --base-model-path nvidia/GR00T-N1.6-3B     --dataset-path /root/.cache/hugginface/lerobot/LSY-lab/stack_without_ft_tact_v4     --embodiment-tag NEW_EMBODIMENT     --modality-config-path finetune-groot/custom_modality_config.py     --num-gpus 1     --output-dir /tmp/so100     --save-total-limit 5     --save-steps 2000     --max-steps 2000     --use-wandb     --global-batch-size 32     --color-jitter-params brightness 0.3 contrast 0.4 saturation 0.5 hue 0.08     --dataloader-num-workers 4
